In [1]:
import yfinance
import stock_utils
import pandas as pd
import numpy as np

from bar import Bar

In [2]:
# fetching liquit tickers, vol > 2M
tickers = stock_utils.fetch_liquid_stocks(vol=2_000_000, min_price=2)
print(f"Fetched {len(tickers)} tickers")

Fetched 617 tickers


In [3]:
#ignored tickers
ignored = stock_utils.get_ignored_symbols() or []
','.join(ignored)

'ACTT,AYR,AMTD,GDI,BPR,LPT,VSLR,ARQL,MDCO,MNTA,PEGI,STML,ERI,HABT,WLH,VTIQ,CY,CSFL,LVGO,CTL,APY,LM,UTX,ZAYO,CAR,MYL,NBL,RARX,AGN,GMHI,FSB,MYOK,AMAG,BWMC,ADSW,PKD,FMCI,AOBC,FII,TLRA,SNH,WBC,DERM,WTR,AVX,GRAF,FSCT,BOLD,HIIQ,IOTS,TRNE,EROS,DRAD,PDLI,GCAP,RUBI,CHK,PE,REAL,CXO'

In [4]:
filtered_tickers = list(set(tickers) - set(ignored))
print(f"Total {len(filtered_tickers)} filtered tickers")

Total 566 filtered tickers


In [5]:
#Historical data for filtered tickers
data = {}
for t in filtered_tickers:
    try:
        s = yfinance.Ticker(t)
        data[t] =  s.history(period='4d')
        data[t].reset_index(inplace=True)
        data[t].columns = data[t].columns.str.lower()
        data[t] = data[t].iloc[::-1,:].reset_index(drop=True)
    except :
        print(t)

- HDS: No data found for this date range, symbol may be delisted
- NTGN: No data found, symbol may be delisted
- BDGE: No data found for this date range, symbol may be delisted
- RMG: No data found for this date range, symbol may be delisted


In [6]:
patterns_list2 = []
for key in data.keys():
    if data[key].empty or len(data[key]) < 2:
        continue
    try:
        bar1 = Bar(
            data[key].loc[0, 'open'],
            data[key].loc[0, 'high'],
            data[key].loc[0, 'low'],
            data[key].loc[0, 'close'],
        )
        bar2 = Bar(
            data[key].loc[1, 'open'],
            data[key].loc[1, 'high'],
            data[key].loc[1, 'low'],
            data[key].loc[1, 'close']
        )
        if bar1  <= bar2:
            patterns_list2.append(key)
    except ValueError as e:
        print(e)
    except Exception as e:
        print(e)

','.join(set(patterns_list2))

Open price must be between High and Low:
 High:30.770000457763672 => Open:29.040000915527344 >= Low:29.110000610351562
Open price must be between High and Low:
 High:31.760000228881836 => Open:30.78849983215332 >= Low:30.860000610351562
Open price must be between High and Low:
 High:103.7300033569336 => Open:100.20999908447266 >= Low:100.2300033569336
Open price must be between High and Low:
 High:14.760000228881836 => Open:14.164999961853027 >= Low:14.1899995803833
Open price must be between High and Low:
 High:139.75999450683594 => Open:137.75 >= Low:137.80999755859375
Open price must be between High and Low:
 High:14.5600004196167 => Open:14.579999923706055 >= Low:12.149999618530273


'AMC,TGI,ENLC,PSEC,MBRX,THCB,BE,LVS,ACTG,ADBE,WELL,INSG,MGY,SSYS,DVN,DT,NBEV,SRNE,Z,USFD,VRTX,EGHT,QEP,PACB,BA,TXN,EXPE,AHT,MPC,BWA,TYME,JAGX,PG,MO,BX,AMD,CNC,MUR,LLNW,FIS,MIK,IVR,MSFT,FSLY,DKS,XEL,EMR,TWO,BNGO,JNPR,DAL,PLD,CNK,NVTA,GEO,CNP,CNX,BLNK,HAL,CLNE,BGCP,PBF,POLA,NVDA,PLNT,PAA,TSLA,WORK,KERN,HES,IZEA,DRRX,EPD,CDEV,WBA,SABR,CCC,MMP,DBX,AQUA,SONO,WY,DDD,QRVO,SWN,PEI,IPG,KOS,NVAX,MTCH,PEIX,NIO,PFE,AMZN,WDC,FE,CVS,AM,CLVS,CCL,AMGN,CL,CPB,ISBC,KNX,VLO,WTRH,CPE,VXRT,PK,TIGR,UPS,CVX,COP,ZYNE,VTNR,SEEL,IVZ,IEA,KR,SLB,OVID,GT,ATUS,BLMN,SCHW,ARMK,KO,MRNA,MU,ALNA,MNKD,FB,ELAN,FTV,F,CRMD,FOLD,PCAR,NLSN,CRM,SQ,RIGL,TMO,NCLH,EVFM,SLM,AA,LC,LOW,AMRS,SKT,KMI,XELA,UAL,AQB,DARE,EXPR,ALXN,COTY,KODK,CRBP,COG,NWL,MARK,MXIM,LITE,X,GSM,UA,OSTK,SYY,SUNW,SIRI,PAGP,FTI,GRUB,DAN,HFC,BDN,MPLX,AGNC,SPR,M,FANG,SLS,ZM,CAG,ADI,AIV,GLUU,PLUG,HRL,TTI,SINO,ACRX,NLY,CAT,NOV,MRO,BAC,VST,GM,HCA,DHR,PSTG,FCX,CRK,SNAP,FCEL,OPTT,EQT,FEYE,HYRE,PHUN,AAL,WMB,CHWY,UAA,VTR,ICLK,HMHC,TMUS,HOG,CCI,JEF,PPL,WFC,ETRN,ALB,HPE,L

In [7]:
patterns_list = []
for key in data.keys():
    if data[key].empty or len(data[key]) < 3:
        continue
    try:
        bar1 = Bar(
            data[key].loc[0, 'open'],
            data[key].loc[0, 'high'],
            data[key].loc[0, 'low'],
            data[key].loc[0, 'close'],
        )
        bar2 = Bar(
            data[key].loc[1, 'open'],
            data[key].loc[1, 'high'],
            data[key].loc[1, 'low'],
            data[key].loc[1, 'close'],
        )
        bar3 = Bar(
            data[key].loc[2, 'open'],
            data[key].loc[2, 'high'],
            data[key].loc[2, 'low'],
            data[key].loc[2, 'close'],
        )
        if bar1 <= bar2 <= bar3:
            patterns_list.append(key)
    except ValueError as e:
        print(e)
        
','.join(set(patterns_list))

Open price must be between High and Low:
 High:30.770000457763672 => Open:29.040000915527344 >= Low:29.110000610351562
Open price must be between High and Low:
 High:31.760000228881836 => Open:30.78849983215332 >= Low:30.860000610351562
Open price must be between High and Low:
 High:103.7300033569336 => Open:100.20999908447266 >= Low:100.2300033569336
Open price must be between High and Low:
 High:14.760000228881836 => Open:14.164999961853027 >= Low:14.1899995803833
Open price must be between High and Low:
 High:139.75999450683594 => Open:137.75 >= Low:137.80999755859375
Open price must be between High and Low:
 High:14.5600004196167 => Open:14.579999923706055 >= Low:12.149999618530273


'AMC,PSEC,MBRX,THCB,ADBE,LVS,INSG,SRNE,Z,VRTX,EGHT,EXPE,MPC,MO,JAGX,PG,AMD,CNC,LLNW,FIS,MIK,FSLY,DKS,XEL,EMR,TWO,BNGO,JNPR,CNK,BLNK,PAA,PLNT,TSLA,WORK,KERN,DRRX,CDEV,WBA,MMP,QRVO,SWN,PFE,FE,WDC,CL,CLVS,VXRT,UPS,GT,KO,FB,ELAN,FTV,CRMD,FOLD,NLSN,CRM,SLM,KMI,LOW,SKT,AQB,EXPR,ALXN,CRBP,NWL,LITE,X,GSM,SIRI,PAGP,MPLX,SPR,CAG,AIV,GLUU,HRL,TTI,NLY,ACRX,CAT,MRO,FCX,CRK,FCEL,EQT,PHUN,WMB,HOG,TMUS,PPL,ETRN,LXRX,MOS,NUAN,VICI,BEN,ALRN,INFN,BCRX,TAL,K'

In [8]:
patterns_list3 = []
for key in data.keys():
    if data[key].empty or len(data[key]) < 3:
        continue
    try:
        bar1 = Bar(
            data[key].loc[0, 'open'],
            data[key].loc[0, 'high'],
            data[key].loc[0, 'low'],
            data[key].loc[0, 'close'],
        )
        bar2 = Bar(
            data[key].loc[1, 'open'],
            data[key].loc[1, 'high'],
            data[key].loc[1, 'low'],
            data[key].loc[1, 'close'],
        )
        bar3 = Bar(
            data[key].loc[2, 'open'],
            data[key].loc[2, 'high'],
            data[key].loc[2, 'low'],
            data[key].loc[2, 'close'],
        )
        if (bar3.range / bar2.range >= 1.7) and (bar2.range / bar1.range) >= 0.95:
            patterns_list3.append(key)
    except ValueError as e:
        print(e)
        
','.join(patterns_list3)

Open price must be between High and Low:
 High:30.770000457763672 => Open:29.040000915527344 >= Low:29.110000610351562
Open price must be between High and Low:
 High:31.760000228881836 => Open:30.78849983215332 >= Low:30.860000610351562
Open price must be between High and Low:
 High:103.7300033569336 => Open:100.20999908447266 >= Low:100.2300033569336
Open price must be between High and Low:
 High:14.760000228881836 => Open:14.164999961853027 >= Low:14.1899995803833
Open price must be between High and Low:
 High:139.75999450683594 => Open:137.75 >= Low:137.80999755859375
Open price must be between High and Low:
 High:14.5600004196167 => Open:14.579999923706055 >= Low:12.149999618530273


'VBIV,EXPE,VXRT,EXPR,NWL,LITE,SIRI,PHUN,BEN,TAL,ABT,PFE,KO,GSM,NEE,AMC,IRM,FOLD,CAT,OPK,HOG,MBRX,BNGO,GPK,CRBP,HRL,BCRX'

# HOMEWORK STOCKS

In [9]:
','.join(set(patterns_list+patterns_list2+patterns_list3))

'AMC,TGI,ENLC,PSEC,MBRX,THCB,ADBE,LVS,BE,ACTG,WELL,INSG,MGY,SSYS,DVN,DT,NBEV,VBIV,SRNE,Z,USFD,VRTX,EGHT,QEP,PACB,BA,TXN,EXPE,MPC,AHT,BWA,MO,JAGX,PG,AMD,BX,TYME,CNC,MUR,LLNW,FIS,MIK,IVR,FSLY,DKS,MSFT,XEL,EMR,TWO,BNGO,JNPR,DAL,PLD,CNK,NVTA,GEO,CNP,CNX,BLNK,HAL,IRM,CLNE,BGCP,PBF,POLA,NVDA,PAA,PLNT,TSLA,WORK,KERN,ABT,HES,IZEA,DRRX,EPD,CDEV,WBA,SABR,MMP,CCC,DBX,AQUA,SONO,WY,DDD,QRVO,SWN,PEI,IPG,KOS,NVAX,MTCH,PFE,PEIX,NIO,FE,WDC,CL,CLVS,AMZN,CVS,AM,CCL,AMGN,CPB,ISBC,KNX,VLO,WTRH,CPE,VXRT,PK,TIGR,UPS,CVX,COP,ZYNE,VTNR,SEEL,IVZ,IEA,KR,SLB,GT,OVID,ATUS,BLMN,SCHW,ARMK,KO,MRNA,MU,ALNA,GPK,FB,MNKD,ELAN,FTV,F,CRMD,FOLD,PCAR,NLSN,CRM,SQ,RIGL,TMO,NCLH,EVFM,SLM,KMI,AA,LOW,LC,SKT,AMRS,XELA,UAL,AQB,DARE,EXPR,ALXN,COTY,KODK,CRBP,NWL,COG,MARK,MXIM,LITE,X,GSM,UA,OSTK,SYY,SUNW,SIRI,PAGP,FTI,GRUB,DAN,HFC,BDN,MPLX,SPR,AGNC,M,FANG,SLS,ZM,CAG,ADI,NEE,AIV,GLUU,HRL,PLUG,TTI,NLY,ACRX,SINO,CAT,MRO,NOV,BAC,VST,GM,HCA,DHR,PSTG,FCX,CRK,SNAP,FCEL,OPTT,OPK,EQT,FEYE,HYRE,PHUN,AAL,WMB,CHWY,UAA,VTR,ICLK,HOG,TMUS,HMHC,CCI,J